In [ ]:
!pip install langgraph

In [2]:
from langgraph.graph import StateGraph, END
from typing import TypedDict, List, Literal

class JobDescriptionState(TypedDict):
    job_title: str
    key_requirements: List[str]
    draft_description: str
    iteration_count: int
    evaluation_score: int
    evaluation_feedback: str
    final_description: str

def create_job_description(state: JobDescriptionState) -> dict:
    iteration = state["iteration_count"]
    print(f"Creating job description (Iteration {iteration + 1})...")

    if iteration == 0:
        draft = f"Job Title: {state['job_title']}\n\nWe're looking for someone great!"
    else:
        feedback = state["evaluation_feedback"]
        draft = f"Job Title: {state['job_title']}\n\nAbout the Role:\nAs a {state['job_title']}, you will succeed.\n\n[Improved based on: {feedback}]"

    return {"draft_description": draft, "iteration_count": iteration + 1}

def evaluate_job_description(state: JobDescriptionState) -> dict:
    print("Evaluating job description...")
    draft = state["draft_description"]
    score = 0
    feedback = []

    if "About the Role" in draft:
        score += 50
    else:
        feedback.append("Missing 'About the Role' section")

    if len(draft) > 100:
        score += 50
    else:
        feedback.append("Description is too short")

    final_feedback = "; ".join(feedback) if feedback else "Looks good!"
    print(f"Score: {score}/100 | Feedback: {final_feedback}")
    return {"evaluation_score": score, "evaluation_feedback": final_feedback}

def finalize_description(state: JobDescriptionState) -> dict:
    print("Job description approved!")
    return {"final_description": state["draft_description"]}

def should_continue_refining(state: JobDescriptionState) -> Literal["refine", "approve"]:
    if state["evaluation_score"] >= 100 or state["iteration_count"] >= 3:
        return "approve"
    else:
        return "refine"

workflow = StateGraph(JobDescriptionState)
workflow.add_node("create", create_job_description)
workflow.add_node("evaluate", evaluate_job_description)
workflow.add_node("finalize", finalize_description)

workflow.set_entry_point("create")
workflow.add_edge("create", "evaluate")
workflow.add_conditional_edges(
    "evaluate",
    should_continue_refining,
    {"refine": "create", "approve": "finalize"}
)
workflow.add_edge("finalize", END)

app = workflow.compile()

initial_state = {
    "job_title": "Senior Python Developer",
    "key_requirements": ["5+ years Python", "AWS"],
    "iteration_count": 0,
}
result = app.invoke(initial_state)

print("\n" + "="*60 + "\nFINAL JOB DESCRIPTION:\n" + "="*60)
print(result["final_description"])

Creating job description (Iteration 1)...
Evaluating job description...
Score: 0/100 | Feedback: Missing 'About the Role' section; Description is too short
Creating job description (Iteration 2)...
Evaluating job description...
Score: 100/100 | Feedback: Looks good!
Job description approved!

FINAL JOB DESCRIPTION:
Job Title: Senior Python Developer

About the Role:
As a Senior Python Developer, you will succeed.

[Improved based on: Missing 'About the Role' section; Description is too short]
